# Исследование опроса клиентов телекоммуникационной компании

**Задачи проекта:**
    
   Исследование данных с результатами опроса, отражающего текущий уровень потребительской лояльности (NPS) клиентов телекоммуникационной компании.

**Описание проекта:**
    
   Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.
   
Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».
Компания провела опрос и попросила подготовить дашборд с его итогами. Итоги необходимо оформить в виде презентации.

Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite.

**План работы:**

   1. Загрузка библиотек и подключение к базе
   2. Выгрузка данных с помощью SQL-запроса
   3.Создание дашборда в Tableau
   4.Презентация итогов исследования

## Загрузка библиотек и подключение к базе

In [1]:
#импортируем необходимые библиотеки
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
#подключение к базе
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

## Выгрузка данных с помощью SQL-запроса

In [3]:
#выгрузка данных с помощью SQL-запроса
query = """
    SELECT u.user_id AS user_id,
    u.lt_day AS lt_day,
       CASE
           WHEN u.lt_day <= 365 THEN 'new'
           ELSE 'old'
       END is_new,
    u.age AS age,
       CASE
           WHEN u.gender_segment == 1 THEN 'женщина'
           WHEN u.gender_segment == 0 THEN 'мужчина'
           ELSE 'Неизвестно'
       END gender_segment,
    u.os_name AS os_name,
    u.cpe_type_name AS cpe_type_name,
    l.country AS country,
    l.city AS city,
    SUBSTRING(ags.title, 3) age_segment,
    SUBSTRING(ts.title, 3) traffic_segment,
    SUBSTRING(ls.title, 3) lifetime_segment,
    u.nps_score AS nps_score,
       CASE
           WHEN nps_score >= 9 THEN 'сторонники'
           WHEN nps_score <= 8 AND nps_score >= 7 THEN 'нейтралы'
           WHEN nps_score <= 6 THEN 'критики'
       END nps_group 
FROM user AS u LEFT JOIN location AS l ON u.location_id=l.location_id
LEFT JOIN age_segment AS ags ON ags.age_gr_id=u.age_gr_id
LEFT JOIN traffic_segment AS ts ON ts.tr_gr_id=u.tr_gr_id
LEFT JOIN lifetime_segment AS  ls ON ls.lt_gr_id=u.lt_gr_id 
"""   

In [4]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,old,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,сторонники
1,A001WF,2344,old,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,сторонники
2,A003Q7,467,old,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,сторонники


In [5]:
#таблицу сохраним как CSV-файл
df.to_csv('telecomm_csi_tableau.csv', index=False)

## Создание дашборда в Tableau:

https://public.tableau.com/app/profile/.38192900/viz/telecomm_csi_tableau_16758884736000/Dashboard2?publish=yes

## Презентация итогов исследования:

https://disk.yandex.ru/d/aYnzHWAHxgVNow